In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql
import pandas as pd
import re
from datetime import datetime, timedelta

In [3]:
#DB 접속
def connection():
    conn = pymysql.connect(
    host = 'localhost',    
    port = 3306,
    user = 'root',
    passwd = 'root&*',
    db = 'zoomdb'
    )
    return conn

In [4]:
#DB 접속 해제
def close(input_conn):
    input_conn.commit()
    input_conn.close()

In [6]:
#0. 메인 메뉴
def menu():
    print('*' * 72)
    print(' 1. 회원가입 | 2. 로그인 | 3. zoom 날짜 등록 | 4. 스터디 모임 찾기 | 5. 게시판 ')
    print(' 프로그램 종료: q')
    choice = input(' 선택: ')

    return choice

In [7]:
#0-1. 게시판 메뉴
def board_menu():
    print(' 1. QnA 등록  |  2. QnA 조회  |  3. 익명글 등록  |  4. 익명 게시판 조회 ')
    board_choice = input(' 선택: ')
    return board_choice

In [8]:
#1. 회원가입
def join():
    conn = connection()
    print('<<< 회원가입 >>>')
    final_email = check_email()
    final_passwd = check_passwd()
    final_name = check_name()
    st_group = input("스터디조: ")
    final_ph = check_ph()

    query = '''insert into student values(null, %s, %s, %s, %s, %s)'''
    cur = conn.cursor()
    cur.execute(query, (final_email, final_passwd, final_name, st_group, final_ph))
    cur.close()
    
    close(conn)

In [9]:
#1-1. 이메일 유효성 검사
def check_email():
    while True:
        st_email = input("이메일 주소: ")
        valid_email = re.match('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', st_email)

        if not valid_email:
            print('유효하지 않은 이메일 주소입니다.')
            print('다시 작성해 주세요.')
        else: break
    
    return st_email

In [10]:
#1-2. 비밀번호 유효성 검사
def check_passwd():
    while True:
        st_passwd = input("비밀번호: ")
        if st_passwd == '': print('비밀번호는 필수 입력 입니다.')
        elif len(st_passwd) < 4: print('4자리 이상 입력해 주세요.')
        else: break
    return st_passwd

In [11]:
#1-3. 이름 유효성 검사
def check_name():
    while True:
        st_name = input('이름: ')
        if st_name == '': print('이름은 필수 입력 입니다.')
        else: break
    return st_name

In [12]:
#1-4. 휴대전화번호 유효성 검사
def check_ph():
    while True:
        st_ph = input("연락처(- 제외 11자리):  ")
        ph1 = st_ph[0:3]
        ph2 = st_ph[3:7]
        ph3 = st_ph[7:]

        if ph1 == '010' or ph1 == '011':
            if len(ph2) < 5 and re.match('[0-9]', ph2):
                if len(ph3) < 5 and re.match('[0-9]', ph3): break
                else: 
                    print('유효하지 않은 연락처입니다.')
                    print('다시 작성해 주세요.')
                    continue
            else: 
                print('유효하지 않은 연락처입니다.')
                print('다시 작성해 주세요.')
                continue
        else:
            print('유효하지 않은 연락처입니다.')
            print('다시 작성해 주세요.')
            continue

    return st_ph

In [13]:
#2. 로그인
def signin():
    while True:
        conn = connection()
        print('*' * 32, 'log in', '*' * 32)
        login_email = input('이메일 주소 입력: ')
        login_pw = input('비밀번호 입력: ')

        query = '''
        select st_id from student 
        where st_email = %s and st_passwd = %s
        '''
        cur = conn.cursor(pymysql.cursors.DictCursor)
        cur.execute(query, (login_email, login_pw))
        result = cur.fetchone()
        
        if result is None: print('이메일 주소 또는 비밀번호가 잘못 입력되었습니다.')
        else: 
            st_id = result['st_id']
            print('로그인 완료')
            break
    cur.close()
    close(conn)
    return st_id

In [14]:
#3. zoom 날짜 등록
def zoom_date(id_param):
    conn = connection()

    t = datetime.today().strftime("%Y-%m-%d") 
    w = (datetime.today() + timedelta(days = 5)).strftime("%Y-%m-%d")
    print('날짜:', t, '~', w)
    print('zoom 참석일과 스터디 희망여부를 적어주세요.')

    zoom_date_dict = {}
    cnt = 0
    while cnt < 3:
        cnt += 1
        zoom_date = input(str(cnt) + '번째 zoom_date 날짜를 입력하세요. ex)01-01 ')
        attend = input('참석여부를 입력하세요. (y/n) ')
        zoom_date_dict [zoom_date] = attend

    for key, value in zoom_date_dict.items():
        query = '''insert into zoom_list(st_id, zoom_date, study) 
                   values(%s, %s, %s)'''
        cur = conn.cursor()
        cur.execute(query, (id_param, key, value))

    cur.close()
    close(conn)

In [15]:
#4. 날짜 선택
def date_choice():
    conn = connection()
    today = datetime.today() + timedelta(days = 6) 
    t_lst = []
    for i in range(5):
        t_lst.append(today.strftime("%Y-%m-%d"))
        today += timedelta(days = 1)
        
    print('번호를 선택하세요.')
    for idx, day in enumerate(t_lst):
        print(idx + 1, '번: ', day)
    want = int(input())

    if want == 1: day = t_lst[0][5:]
    elif want == 2: day = t_lst[1][5:]
    elif want == 3: day = t_lst[2][5:]
    elif want == 4: day = t_lst[3][5:]
    else: day = t_lst[4][5:]
    print('*' * 20, day, 'zoom으로 수업 듣는 명단', '*' * 20)

    sql = '''
        select st_name 이름, st_email 이메일, st_ph 연락처, st_group 조 from student s
        join zoom_list z on s.st_id = z.st_id
        where z.study = 'y' and z.zoom_date = %s
        '''
    dictCur = conn.cursor(pymysql.cursors.DictCursor)
    dictCur.execute(sql, (day))
    dictResult = dictCur.fetchall()
    dictCur.close()
    close(conn)
    
    df = pd.DataFrame(dictResult)
    if df.shape == (0, 0): print('no data')
    else: print(df)

In [16]:
#5-1. Qna 게시판 등록
def qna(id_param):
    conn = connection()
    print('*'*30, 'Q&A 게시판', '*'*30)
    title = input('글의 제목을 입력하세요. ')
    contents = input('글의 내용을 입력하세요. ')
    
    query = '''insert into qna values(null, %s, %s, %s, default)'''
    cur = conn.cursor()
    cur.execute(query, (id_param, title, contents))
    cur.close()
    close(conn)
    
    return print('글 등록이 완료되었습니다.')

In [17]:
#5-2. Qna 조회
def qna_view():
    conn = connection()
    print('*'*30, 'Q&A 게시판', '*'*30)
    query = '''select st_name 작성자, title 제목, contents 내용, 
               date_format(written, '%m-%d') 작성일 from qna''' 
    dictCur = conn.cursor(pymysql.cursors.DictCursor)
    dictCur.execute(query)
    dictResult = dictCur.fetchall()
    dictCur.close()
    close(conn)

    df = pd.DataFrame(dictResult)
    print(df)

In [18]:
#5-3. 익명 게시판 등록
def add_descript():
    conn = connection()
    title = input('제목: ')
    descript = input('내용: ')
    sql = '''
          insert no_name_board
          values(null, default, %s, %s)
          '''
    cur = conn.cursor()
    cur.execute(sql, (title, descript))
    cur.close()
    close(conn)
    
    return print('입력이 완료되었습니다.')

In [19]:
#5-4. 익명 게시판 조회
def print_board():
    conn = connection()
    sql = '''
            select title 제목, descript 내용, date_format(write_day, '%m-%d') 작성일
            from no_name_board
          '''
    dictCur = conn.cursor(pymysql.cursors.DictCursor)
    dictCur.execute(sql)
    dictResult = dictCur.fetchall()
    dictCur.close()
    close(conn)
    
    df = pd.DataFrame(dictResult)
    print(df)

In [20]:
while True:
    choice = menu()
    if choice == 'q': break
    elif choice == '1': join()
    elif choice == '2': st_id_output = signin()
    elif choice == '3': zoom_date(st_id_output)
    elif choice == '4':
        date_choice()
    else: 
        input_choice = board_menu()
        if input_choice == '1': qna(st_id_output)
        elif input_choice == '2': qna_view()
        elif input_choice == '3': add_descript()
        elif input_choice == '4': print_board()


************************************************************************
 1. 회원가입 | 2. 로그인 | 3. zoom 날짜 등록 | 4. 스터디 모임 찾기 | 5. 게시판 
 프로그램 종료: q

 선택: 4

번호를 선택하세요.
1 번:  2021-04-19
2 번:  2021-04-20
3 번:  2021-04-21
4 번:  2021-04-22
5 번:  2021-04-23
1

******************** 04-19 zoom으로 수업 듣는 명단 ********************
    이름                   이메일          연락처   조
0  문현규  gusrb4805@icloud.com  01011115555  1조
1  문현규  gusrb4805@icloud.com  01011115555  1조
2  문현규  gusrb4805@icloud.com  01011115555  1조
3  배예진   byjin0229@gmail.com  01037845210  1조
4  김강산    ka12sa3@icloud.com  01037894561  5조
5  권소희    sohe0214@naver.com  01012223434  3조


************************************************************************
 1. 회원가입 | 2. 로그인 | 3. zoom 날짜 등록 | 4. 스터디 모임 찾기 | 5. 게시판 
 프로그램 종료: q

 선택: q

